### Import required packages

In [1]:
import pandas as pd
import arviz as az

### Read in results from the two different methods

In [ ]:
idata_iid = az.from_netcdf("../data/idata_iid.nc")
summary_df = pd.read_csv("../data/iid_summary.csv", index_col=0)
# print(summary_df)

In [ ]:
'''idata_ar1 = az.from_netcdf("../data/idata_ar1.nc")
summary_ar1 = pd.read_csv("../data/ar1_summary.csv", index_col=0)
# print(summary_ar1)'''

In [ ]:
df_FinBERT = pd.read_csv('../data/sp500_headlines_2008_2024_FinBERT.csv')